In [1]:
import os

In [2]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions'

entity

In [7]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Evaluation_Config:
    test_faces_dir: Path
    represent_file:Path
    test_dir:Path
    model_name: str
    face_detector_backend: str
    target_size: tuple
    align: bool
    enforce_detection: bool
    expand_percentage: int
    normalization:str
    silent: bool
    gray_scale: bool
    distance_matrix: str
    threshold:int



src configaration management

In [14]:
from face_recognitions.constants import *
from face_recognitions.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_YAML_PATH,
                 params_filepath=PARAMS_YAML_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_eval_config(self)->Evaluation_Config:
        config=self.config.evaluation
        represent_file=self.config.train_model.represent_file
        params=self.params
        create_directories([config.test_dir])

        eval_config=Evaluation_Config(
            test_dir=config.test_dir,
            represent_file=represent_file,
            test_faces_dir=config.test_faces_dir,
            model_name= params.MODEL_NAME,
            face_detector_backend=params.FACE_DETECTOR_BACKEND,
            target_size=params.TARGET_SIZE,
            align=params.ALIGN,
            enforce_detection=params.ENFORCE_DETECTION,
            expand_percentage=params.EXPAND_PERCENTAGE,
            normalization=params.NORMALIZATION,
            silent=params.SILENT,
            gray_scale=params.GRAY_SCALE,
            distance_matrix=params.DISTANCE_MATRIX,
            threshold=params.THRESHOLD
        )

        return eval_config

Evaluation model Component

In [30]:
from face_recognitions.utils.common import read_yaml,list_of_image_path

ImportError: cannot import name 'list_of_image_path' from 'face_recognitions.utils.common' (d:\develop\pipline\face_recognitions\src\face_recognitions\utils\common.py)

In [43]:
from face_recognitions import logger
# from face_recognitions.utils.common import list_of_image_path
from deepface.modules import representation, detection, modeling, verification
from deepface.models.FacialRecognition import FacialRecognition

from pathlib import Path

import os
import pickle
import pandas as pd
from typing import List, Optional, Tuple
import time

class Evaluation:
    def __init__(self,config:Evaluation_Config):
        self.config=config
    
    def face_name(self,path:Path)->str:
        return os.path.basename(os.path.dirname(path))
    
    def list_of_image_path(self,path:Path)->List[Path]:

        """
        List images in a given path
        Args:
            path (str): path's location
        Returns:
            images (list): list of exact image paths
        """
        images = []
        for r, _, f in os.walk(path):
            for file in f:
                if file.lower().endswith((".jpg", ".jpeg", ".png")):
                    exact_path = os.path.join(r, file)
                    images.append(exact_path)
        return images
    
    def find(self,
             find_img_path:Path,
             represent_model_df:pd.DataFrame,
             target_size:Tuple[int, int],
             threshold: Optional[float] = None,
             )->List[pd.DataFrame]:
        tic = time.time()

        config=self.config
        distance_metric=config.distance_matrix

        # img path might have more than once face
        source_objs = detection.extract_faces(
            img_path=find_img_path,
            target_size=target_size,
            detector_backend=config.face_detector_backend,
            grayscale=config.gray_scale,
            enforce_detection=config.enforce_detection,
            align=config.align,
            expand_percentage=config.expand_percentage,
        )

        resp_obj = []

        for source_obj in source_objs:
            source_img = source_obj["face"]
            source_region = source_obj["facial_area"]
            target_embedding_obj = representation.represent(
                img_path=source_img,
                model_name=config.model_name,
                enforce_detection=config.enforce_detection,
                detector_backend="skip",
                align=config.align,
                normalization=config.normalization,
            )

            target_representation = target_embedding_obj[0]["embedding"]

            result_df = represent_model_df.copy()  # df will be filtered in each img
            result_df["source_x"] = source_region["x"]
            result_df["source_y"] = source_region["y"]
            result_df["source_w"] = source_region["w"]
            result_df["source_h"] = source_region["h"]

            distances = []
            for _, instance in represent_model_df.iterrows():
                source_representation = instance[f"{config.model_name}_representation"]
                if source_representation is None:
                    distances.append(float("inf")) # no representation for this image
                    continue

                target_dims = len(list(target_representation))
                source_dims = len(list(source_representation))
                if target_dims != source_dims:
                    raise ValueError(
                        "Source and target embeddings must have same dimensions but "
                        + f"{target_dims}:{source_dims}. Model structure may change"
                        + " after pickle created. Delete the {file_name} and re-run."
                    )

                if distance_metric == "cosine":
                    distance = verification.find_cosine_distance(
                        source_representation, target_representation
                    )
                elif distance_metric == "euclidean":
                    distance = verification.find_euclidean_distance(
                        source_representation, target_representation
                    )
                elif distance_metric == "euclidean_l2":
                    distance = verification.find_euclidean_distance(
                        verification.l2_normalize(source_representation),
                        verification.l2_normalize(target_representation),
                    )
                else:
                    raise ValueError(f"invalid distance metric passes - {distance_metric}")

                distances.append(distance)

                # ---------------------------
            target_threshold = threshold or verification.find_threshold(config.model_name, distance_metric)

            result_df["threshold"] = target_threshold
            result_df["distance"] = distances

            result_df = result_df.drop(columns=[f"{config.model_name}_representation"])
            # pylint: disable=unsubscriptable-object
            result_df = result_df[result_df["distance"] <= target_threshold]
            result_df = result_df.sort_values(by=["distance"], ascending=True).reset_index(drop=True)
            resp_obj.append(result_df)

    # -----------------------------------

        if not config.silent:
            toc = time.time()
            logger.info(f"find function duration {toc - tic} seconds")

        return resp_obj
   
    def Eval(self):
        tic = time.time()

        config=self.config
         # -------------------------------
        if os.path.isdir(config.test_faces_dir) is not True:
            raise ValueError("Passed test_faces_dir does not exist!")
        
        test_list_image_path=self.list_of_image_path(config.test_faces_dir)

        # -------------------------------
        if  not os.path.exists(config.represent_file):
            raise ValueError("Passed represent_file does not exist!")

        model: FacialRecognition = modeling.build_model(config.model_name)
        target_size = model.input_shape

        # ---------------------------------------

        df_cols = [
            "identity",
            f"{config.model_name}_representation",
            "target_x",
            "target_y",
            "target_w",
            "target_h",
            ]
        
        # Load the representations from the pickle file
        with open(config.represent_file, "rb") as f:
            representations = pickle.load(f)
    
            # Should we have no representations bailout
        if len(representations) == 0 or len(representations)<0:
            raise ValueError(f"Retrain representation - size of representation {len(representations)}")
        
        # now, we got representations for facial database
        df = pd.DataFrame(
            representations,
            columns=df_cols,
        )

        evaluation_test_faces_result=[]
        
        for image_path in test_list_image_path[:10]:
            result=self.find(
                find_img_path=image_path,
                represent_model_df=df,
                target_size=target_size,
            )
            print(image_path)
            evaluation_test_faces_result.append(result)
        
        # -----------------------------------

        if not config.silent:
            toc = time.time()
            logger.info(f"find function duration {toc - tic} seconds")

        return evaluation_test_faces_result




    

In [44]:
try:
    obj=ConfigurationManager()
    aa=obj.get_eval_config()
    bb=Evaluation(config=aa)

    df=bb.Eval()
except Exception as e:
    raise e

[2024-02-28 17:13:03,959: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-28 17:13:03,959: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-28 17:13:03,971: INFO: common: created directory at: artifacts]
[2024-02-28 17:13:03,975: INFO: common: created directory at: artifacts/test]
1/1 [==============================] - 0s 33ms/step
[2024-02-28 17:13:07,323: INFO: 2575025493: find function duration 3.1305601596832275 seconds]
artifacts/data_ingestion/dataset/test\102\7755.jpg
1/1 [==============================] - 0s 33ms/step
[2024-02-28 17:13:10,262: INFO: 2575025493: find function duration 2.939180612564087 seconds]
artifacts/data_ingestion/dataset/test\102\8526.jpg
1/1 [==============================] - 0s 31ms/step
[2024-02-28 17:13:13,395: INFO: 2575025493: find function duration 3.1324055194854736 seconds]
artifacts/data_ingestion/dataset/test\102\9988.jpg
1/1 [==============================] - 0s 41ms/step
[2024-02-28 17:13:16,426: INF